# How to Run TARDIS Model Grids

This notebook demonstrates the capabilities of the TARDIS grid. The grid facilitates users running large numbers of TARDIS simulations.

In [1]:
import pandas as pd
import numpy as np
from tardis.io.atom_data import download_atom_data
from tardis.io.atom_data import AtomData

import tardis.grid as grid

Iterations:          0/? [00:00<?, ?it/s]

Packets:             0/? [00:00<?, ?it/s]

In [2]:
# We download the atomic data needed to run a TARDIS simulation
download_atom_data('kurucz_cd23_chianti_H_He')


Atomic Data kurucz_cd23_chianti_H_He already exists in /home/runner/Downloads/tardis-data/kurucz_cd23_chianti_H_He.h5. Will not download - override with force_download=True.


## Creating A Grid

There are 2 ways of creating a TARDIS grid: directly from a dataframe, or by defining a set of axes over which the grid should be defined. In both cases, a config.yml file is required. **Note that for the dataframe, the column names are in the form of valid keys in a tardis Configuration dictionary. For the axes, the keys must similarly be valid tardis Configuration keys.**

In [3]:
# Load a DataFrame
df = pd.read_csv('example_grid.txt')
df

,model.abundances.H,model.abundances.He,model.structure.velocity.start,plasma.initial_t_inner
0,1.0,0.0,10000,5000
1,0.4,0.6,12000,6000
2,0.7,0.3,15000,7000


In [4]:
# Create a tardis grid directly from a dataframe.
grid.TardisGrid(configFile='example.yml', gridFrame=df)

In [5]:
# Create an axes dictionary
axesdict = {'model.structure.velocity.start':np.arange(10000,15000,1000), 
            'model.abundances.He':np.arange(0,1,0.1),
            'model.abundances.H':np.arange(0,1,0.25)}

In [6]:
#Create a tardis grid from an axes dict using the classmethod.
grid.TardisGrid.from_axes(configFile='example.yml', axesdict=axesdict)

## TARDIS Grid Attributes

The TARDIS grid only has 2 attributes. It creates a TARDIS Configuration object from the user specified config.yml file and saves this to the `self.config` attribute. The grid also stores the parameters of every grid point in a Dataframe accessed by `self.grid`

In [7]:
tg = grid.TardisGrid(configFile='example.yml', gridFrame=df)

In [8]:
# The config generated from the user's yml file.
tg.config

{'tardis_config_version': 'v1.0',
 'supernova': {'luminosity_requested': <Quantity 5.679e+41 erg / s>,
  'luminosity_wavelength_start': <Quantity 3481.82000178 Angstrom>,
  'luminosity_wavelength_end': <Quantity 9947.78776065 Angstrom>,
  'time_explosion': <Quantity 10. d>},
 'atom_data': 'kurucz_cd23_chianti_H_He.h5',
 'model': {'structure': {'type': 'specific',
   'velocity': {'start': <Quantity 10000. km / s>,
    'stop': <Quantity 20000. km / s>,
    'num': 20},
   'density': {'type': 'branch85_w7',
    'w7_time_0': <Quantity 0.00023148 d>,
    'w7_rho_0': <Quantity 3.e+29 g / cm3>,
    'w7_v_0': <Quantity 1. km / s>}},
  'abundances': {'type': 'uniform',
   'He': 0.2,
   'H': 0.8,
   'model_isotope_time_0': <Quantity 0. s>}},
 'plasma': {'initial_t_inner': <Quantity 7000. K>,
  'disable_electron_scattering': False,
  'ionization': 'nebular',
  'excitation': 'dilute-lte',
  'radiative_rates_type': 'dilute-blackbody',
  'line_interaction_type': 'macroatom',
  'initial_t_rad': <Quant

In [9]:
# The user specified grid.
tg.grid

,model.abundances.H,model.abundances.He,model.structure.velocity.start,plasma.initial_t_inner
0,1.0,0.0,10000,5000
1,0.4,0.6,12000,6000
2,0.7,0.3,15000,7000


## TARDIS Grid Functionality

The TARDIS Grid provides a variety of functions for using the grid to generate new Configurations, return new SimulationState objects, or directly run simulations using the parameters specified by the grid. This functionality is particularly useful for running large numbers of simulations and easily works with JobArrays where the row_index is the JobArray index.

In [10]:
# Easily get a new TARDIS Configuration object 
# which has the properties of the base self.config
# but with properties modified by a row of the grid.
new_grid = tg.grid_row_to_config(row_index=0);
print("tg.config is the original config:",tg.config.model.abundances)
print("The new config is modified: ",new_grid.model.abundances)

tg.config is the original config: {'type': 'uniform', 'He': 0.2, 'H': 0.8, 'model_isotope_time_0': <Quantity 0. s>}
The new config is modified:  {'type': 'uniform', 'He': np.float64(0.0), 'H': np.float64(1.0), 'model_isotope_time_0': <Quantity 0. s>}


In [11]:
# In case a user needs to make more complicated changes
# to the base TARDIS model (i.e. using parameters that 
# are not valid TARDIS Configuration keys), the grid
# can return a SimulationState object for a given row.
atom_data = AtomData.from_hdf(tg.config.atom_data)
simulation_state = tg.grid_row_to_simulation_state(row_index=0, atomic_data=atom_data)
simulation_state

Number of density points larger than number of shells. Assuming inner point irrelevant


In [12]:
# Users can easily run a full TARDIS simulation
# using the grid.
sim = tg.run_sim_from_grid(row_index=0)

Tabs(height=350, sizing_mode='stretch_width')
    [0] HTML(str, height=300, styles={'overflow-y': 'auto', ...})
    [1] HTML(str, height=300, styles={'overflow-y': 'auto', ...})
    [2] HTML(str, height=300, styles={'overflow-y': 'auto', ...})
    [3] HTML(str, height=300, styles={'overflow-y': 'auto', ...})

TqdmHBox(children=(HTML(value='Iterations:', layout=Layout(width='6%')), FloatProgress(value=0.0, layout=Layou…

TqdmHBox(children=(HTML(value='Packets:\u2007\u2007\u2007', layout=Layout(width='6%')), FloatProgress(value=0.…